In [1]:
import numpy as np
import pandas as pd

In [2]:
tableau = pd.read_csv('tableau.csv')

In [3]:
appsflyer_raw = pd.read_csv('merged_file.csv')

/var/folders/81/8rrb2nqs7t39v9t0vy9jm2s80000gq/T/ipykernel_42246/3071957616.py:1: DtypeWarning: Columns (0,1,9,13,14,15,16,18,20,21,22,23,31,32,33,36,37,39,40,51,53,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  appsflyer_raw = pd.read_csv('merged_file.csv')


In [4]:
appsflyer_raw.shape

(181282, 59)

In [5]:
# data cleaning
appsflyer = appsflyer_raw[['Customer User ID','Install Time', 'Media Source', 'Platform']]
appsflyer = appsflyer.dropna(subset='Customer User ID')
appsflyer.columns = ['customer_user_id','install_time', 'media_source', 'platform']
appsflyer = appsflyer.drop_duplicates(subset='customer_user_id')
appsflyer = appsflyer.fillna({'media_source':'organic'})

In [6]:
# merge data
appsflyer_tableau = pd.merge(appsflyer, tableau, how='left',left_on='customer_user_id', right_on='order_member_uuid')

In [7]:
# convert data type to datetime
appsflyer_tableau.install_time = pd.to_datetime(appsflyer_tableau.install_time)
appsflyer_tableau.order_create_dt_tw = pd.to_datetime(appsflyer_tableau.order_create_dt_tw)
appsflyer_tableau['install_date'] = appsflyer_tableau.install_time.dt.date

In [8]:
# 篩選出每個用戶從安裝日開始，30天內的交易資料
df_filtered = pd.DataFrame()
for uuid, group in appsflyer_tableau.groupby('customer_user_id'):
    group_filtered = group[(group['order_create_dt_tw'] >= group['install_date']) & 
                           (group['order_create_dt_tw'] <= group['install_date'] + pd.Timedelta(days=30))]
    df_filtered = pd.concat([df_filtered, group_filtered])

In [9]:
# 將 df_filtered 的交易資料加總至每一個用戶
total_sale_per_user = df_filtered.groupby('customer_user_id').sum(['accounting_price', 'order_profit'])
total_sale_per_user = total_sale_per_user.reset_index()

In [10]:
# 將install, media_source, platform資訊加回去
app_download_sales = pd.merge(appsflyer, total_sale_per_user, how='left', on='customer_user_id')
app_download_sales = app_download_sales.fillna(0)

In [11]:
app_download_sales.install_time = pd.to_datetime(app_download_sales.install_time)

In [12]:
# 篩選需要分析的資料
app_download_sales_selected = app_download_sales[app_download_sales.install_time <= pd.Timestamp('2024-08-07')]

In [13]:
app_download_sales_selected['month'] = app_download_sales_selected.loc[:, 'install_time'].dt.month

/var/folders/81/8rrb2nqs7t39v9t0vy9jm2s80000gq/T/ipykernel_42246/3901327513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_download_sales_selected['month'] = app_download_sales_selected.loc[:, 'install_time'].dt.month


In [14]:
app_download_sales_selected

,customer_user_id,install_time,media_source,platform,accounting_price,order_profit,month
0,42627abe-9b98-408a-a146-148c6eb56529,2024-07-31 23:55:37,googleadwords_int,android,0.00,0.00,7
1,a833a424-5903-436f-8c4c-5984f9905a36,2024-07-30 17:53:19,googleadwords_int,android,71.23,11.83,7
2,96896acd-012d-41eb-ab2a-4db2c9bb4bf3,2024-07-31 23:51:28,googleadwords_int,android,0.00,0.00,7
3,2b4e1f45-3fa2-42d6-af72-5563a2bccd41,2024-07-31 23:48:49,Smart_Banner,android,0.00,0.00,7
4,af46299b-74d0-401f-98de-06580eb7e1a3,2024-07-31 23:48:49,Smart_Banner,android,0.00,0.00,7
...,...,...,...,...,...,...,...
51201,534662c8-d854-486c-a7cd-2129ab4e7c2c,2024-08-01 01:27:20,organic,ios,0.00,0.00,8
51202,6331aeed-a7a7-4147-9fc6-d6e0f8d61fd5,2024-08-01 01:16:21,organic,ios,0.00,0.00,8
51203,ad1ddc6c-68a4-47fa-9615-70ee696da947,2024-08-01 00:42:14,organic,ios,140.40,-5.22,8
51204,62207612-d038-425d-bee5-8ccddc952684,2024-08-01 00:57:25,organic,ios,298.71,42.99,8


In [15]:
profit = app_download_sales_selected.groupby(['platform', 'month', 'media_source']).sum('order_profit')

In [16]:
profit

accounting_price  order_profit
platform month media_source                                  
android  7     Blog                      255.43      17.70000
               Blog_tw                   358.56      57.36960
               CRM_event                 482.07     154.01000
               Email                       0.00       0.00000
               Facebook_ads              238.21      11.13000
...                                         ...           ...
ios      8     jp_promopage                0.00       0.00000
               organic                 43365.80    3464.12839
               rd_mkt_web               5949.57     574.73000
               tw_productpage           1354.34     173.19800
               tw_promopage                0.00       0.00000

[112 rows x 2 columns]

In [17]:
counts = app_download_sales_selected[['platform', 'month', 'media_source']].groupby(
    ['platform', 'month', 'media_source']
).value_counts()

In [18]:
counts

platform  month  media_source  
android   7      Blog               15
                 Blog_tw             3
                 CRM_event          18
                 Email               1
                 Facebook_ads       23
                                  ... 
ios       8      jp_promopage        1
                 organic           885
                 rd_mkt_web         52
                 tw_productpage     16
                 tw_promopage        1
Name: count, Length: 112, dtype: int64

In [19]:
# export to csv
profit.to_csv('profit.csv')
counts.to_csv('counts.csv')